<a href="https://colab.research.google.com/github/amyth18/CS598-Deep-Learning-Final-Project/blob/main/CS598_Deep_Learning_For_Healthcare_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Load Data

In [ ]:
# read data
df = pd.read_csv("/content/drive/My Drive/DLH Final Project/mimic3/NOTEEVENTS.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.head(2)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...


# Data Preprocessing
Need to focus on 2 tables
1. NOTESEVENTS.csv
2. DIAGNOSES_ICD.csv

Join tables by <subject_id, hadm_id>

Construct 2 datasets from "TEXT" field in NOTESEVENTS.csv for each <subject_id, hadm_id> pair (i.e discharge summary for that admission.)

X1, y and X2, y
x1 = sequence of vectors from word2vec 
x2 = sequence of vectors from tf-idf
y = list of icd codes for <subject_id, hadm_id> i.e. diagnosis maded in ICU admission.

Need to focus on 50 and 100 most commonly diagnosed diseases.

## Common data preprocessing
1. Use NLTK + MetaMap to extract only the symptom related entities (how to use MetaMap is unknown atm.)

2. Filter out sections in discharge summaries that are related to symptoms only, ignore others to speed up things.

3. Negative filters (e.g. "no sign of breath problem").

## Data processing for X1 (Word2Vec)
1. Generate Word2Vec embeddings (currently using Gensim) using "TEXT".

## Data processing for X2 (TF-IDF)
1. Generate TF-IDF vector for each symptom entity.

## Data processing for y
1. multi-hot encoding?
 




In [ ]:
import re
import nltk

from nltk.corpus import stopwords
nltk.download('stopwords')

eng_stop_words =  stopwords.words('english')

class MySentences(object):
    def __init__(self, dframe):
        self.dframe = dframe
    
    # TODO: Keeping only alpha numeric characters and spaces for now.
    # need to make this better. Find some good libraries.
    def sanitize_text(self, text):
      test = text.strip()
      text = re.sub(r'\s\s+', ' ', text)
      text = re.sub(r'[^a-zA-z0-9\/\.\?\!\s;,\'\-]', '', text)
      text = re.sub(r'[\.\-\/\?\!;,]', ' ', text)
      text = re.sub(r'[\[\]]', '', text)
      return text

    # TODO: adding some basic checks again need to make it better.
    def sanitize_words(self, sentence):
      return [w for w in sentence if w not in eng_stop_words and not w.isdigit()]

    def __iter__(self):
        for idx in range(len(self.dframe)):
          text = self.sanitize_text(self.dframe["TEXT"][idx])
          yield self.sanitize_words(text.split())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df_icd_codes = pd.read_csv("/content/drive/My Drive/DLH Final Project/mimic3/DIAGNOSES_ICD.csv")

In [ ]:
df_icd_codes.head(2)

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486


In [ ]:
counts = df_icd_codes["ICD9_CODE"].value_counts().head(50)
top_icd_codes = counts.index.to_list()
df_admissions_with_top_diseases = df_icd_codes[df_icd_codes["ICD9_CODE"].isin(top_icd_codes)]
df_unique_admissions_with_top_diseases = df_admissions_with_top_diseases[["SUBJECT_ID", "HADM_ID"]].drop_duplicates()

In [ ]:
len(df_unique_admissions_with_top_diseases)

55078

In [ ]:
filtered_df = pd.merge(df, df_unique_admissions_with_top_diseases, on=["SUBJECT_ID", "HADM_ID"])

In [ ]:
filtered_df_disc_sum = filtered_df[filtered_df["CATEGORY"] == 'Discharge summary'].reset_index()
print(len(filtered_df_disc_sum))
print(filtered_df_disc_sum.head(5))

Test code for sentence generator

In [ ]:
sgen = MySentences(filtered_df_disc_sum)
sentences = list()
for idx, s in enumerate(sgen):
  print(s)
  if idx == 1:
    break

Word2Vec training using gensim.

In [ ]:
import gensim
# NOTE: commenting this part so that we dont run this by mistake.
# sgen = MySentences(filtered_df_disc_sum) # a memory-friendly iterator
# model = gensim.models.Word2Vec(sgen, min_count=5, workers=4, sample=1e-05)
# model.save("/content/drive/My Drive/DLH Final Project/mimic3/word2vec.model")

In [ ]:
!ls -lh "/content/drive/My Drive/DLH Final Project/mimic3/"

total 3.9G
-rw------- 1 root root  19M Mar 19  2019 DIAGNOSES_ICD.csv
-rw------- 1 root root 3.8G Jun 14  2019 NOTEEVENTS.csv
-rw------- 1 root root  78M Apr 13 08:14 word2vec.model


In [ ]:
# multi-hot encoding for icd_codes for a given admission

series_admissions_with_top_diseases_grouped = \
df_admissions_with_top_diseases.groupby(['SUBJECT_ID', 
                                         'HADM_ID'])['ICD9_CODE'].apply(list)

df_admissions_with_top_diseases_grouped = \
series_admissions_with_top_diseases_grouped.to_frame().reset_index()

In [ ]:
sorted_top_icd_codes = sorted(top_icd_codes)
icd_code_to_idx = dict((k, v) for v, k in enumerate(sorted_top_icd_codes))

In [ ]:
# new col to be added to dataframe
multi_hot_ecoding_col = list()
for idx in range(len(df_admissions_with_top_diseases_grouped)):
  icd_codes = df_admissions_with_top_diseases_grouped.iloc[idx]['ICD9_CODE']
  encoding = [0] * 50
  for code in icd_codes:
    encoding[icd_code_to_idx[code]] = 1    
  multi_hot_ecoding_col.append(encoding)

df_admissions_with_top_diseases_grouped['ICD9_CODE_ENCODED'] = multi_hot_ecoding_col

In [ ]:
print(len(df_admissions_with_top_diseases_grouped))
print(len(filtered_df_disc_sum))

# TODO: there are more records in filtered_df_disc_sum than 
# df_admissions_with_top_diseases_grouped, why??

55078
55988


In [ ]:
# create our final dataset we will be working
# join df_admissions_with_top_diseases_grouped and filtered_df_disc_sum